In [12]:
import dataset_models
import imp
imp.reload(dataset_models)

import pandas as pd
from dataset_models import Umi, session, db, Dataset, GeneGoTerm, UmiGeneId, GoTerm, NcbiGene, Segment

rsq = pd.read_sql_query
sq = session.query

from sqlalchemy import desc, asc, func, distinct
import pandas as pd
import numpy as np
import json
from scipy import stats
import os

In [29]:

dsname= "29045011805220233"
getDatasetId =  lambda dsname: int(dsname[:8])

nm = dsname
out_folder = os.path.join(f"/data/dd-analysis/datasets/{nm}/segmentations/")
if not os.path.isdir(out_folder):
        os.makedirs(out_folder)

def winning_pixels_file(dsname):
    out_folder = os.path.join(f"/data/dd-analysis/datasets/{dsname}/segmentations/")
    fn = os.path.join(out_folder,"winning_pixels.csv")
    
def passing_pixels_file(dsname):
    out_folder = os.path.join(f"/data/dd-analysis/datasets/{dsname}/segmentations/")
    fn = os.path.join(out_folder,"passing_pixels.csv")




In [6]:
getDatasetId =  lambda dsname: int(dsname[:8])
umis= rsq(sq(Umi).filter(Umi.dsid==getDatasetId("29045011")).statement,db).rename({"id":"umi_id"},axis="columns").set_index("umi_id")
# initialize segment information
segment_umis = rsq(sq(Umi.dsid,Umi.id.label("umi_id"),Umi.x,Umi.y,Segment.id.label("seg_id")).filter(Umi.dsid==29045011).join(Segment).statement,db)
segs = segment_umis.join(segment_umis.seg_id.value_counts().rename("seg_umi_count"), on = "seg_id").loc[lambda x: x.seg_umi_count>1]
segs_by_id = segs.set_index("seg_id")

#compute normalized distances to cell centers using eigenvectors of the covariance matrix
esys = segs_by_id.groupby("seg_id").apply(lambda g: np.linalg.eig(np.cov(*g[["x","y"]].values.T)))
emeans = segs_by_id[["x","y"]].groupby("seg_id").mean()
proj_dist = segs_by_id.reset_index().set_index("umi_id").apply(
    lambda u:(lambda e: (np.dot(u[["x","y"]] - emeans.loc[u.seg_id] ,e[1][:,0])/(e[0][0]**.5))**2 + \
                        (np.dot(u[["x","y"]] - emeans.loc[u.seg_id] ,e[1][:,1])/(e[0][1]**.5))**2)(esys.loc[u.seg_id]),axis=1)\
    .rename("transformed_dist")


segs_with_dists = segs_by_id.join(proj_dist.reset_index().set_index("umi_id"),on="umi_id")
segs_with_dists = segs_with_dists.join(umis[["total_reads"]],on="umi_id")

combined_series = pd.Series()

i = 0
for k, g in segs_with_dists.groupby(level=0):
    gsubs = g.loc[g.total_reads>0]
    xmin= gsubs.x.mean()- (np.max([gsubs.x.std(),.5]))
    xmax= gsubs.x.mean()+ (np.max([gsubs.x.std(),.5]))
    ymin= gsubs.y.mean()- (np.max([gsubs.y.std(),.5]))
    ymax= gsubs.y.mean()+ (np.max([gsubs.y.std(),.5]))

    #X, Y = np.mgrid[xmin:xmax:500j, ymin:ymax:500j]

    pts = 2
    res = 10**(-1*pts)
    #print(res)
    X, Y = np.round(np.mgrid[np.round(xmin,pts):np.round(xmax,pts):res, np.round(ymin,pts):np.round(ymax,pts):res],pts)
    positions = np.vstack([X.ravel(), Y.ravel()])

    values = np.vstack([gsubs.x,gsubs.y])
    kernel = stats.gaussian_kde(values,weights=np.log(gsubs.total_reads.astype(float).values))#, bw_method=1,)
    Z = np.reshape(kernel.evaluate(positions).T, X.shape)

    df = pd.DataFrame(Z)#/ np.sum(Z)
    df.columns =pd.Series(Y[0,:]).rename("y")
    df.index =pd.Series(X[:,0]).rename("x")

    series = pd.DataFrame(df.stack()).assign(g=k).set_index("g",append=True).iloc[:,0]
    combined_series = pd.concat([series,combined_series])
    
    i+=1
    if i % 10 == 0:
        print(i)

    
combined_series=combined_series.rename("kde_vals")
val_sorted  = combined_series.sort_values(0,ascending=False).sort_index(level=[0,1], sort_remaining=False)

#threshold kde distributions to create "passing" pixels considered to be hits for each cell segment
passing_pixels = val_sorted.groupby("g").apply(lambda x: x.loc[x> 1]).reset_index(level=0,drop=True)

#argmax thresholded kde distributions to create bitmasks indicating which cell will have precendence in each pixel
best_seg = val_sorted.reset_index(level = "g").loc[lambda x: ~x.index.duplicated()].g
best_val = val_sorted.reset_index(level = "g").loc[lambda x: ~x.index.duplicated()].kde_vals
winning_pixels = best_seg.loc[best_val > 1]


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [8]:

sns.scatterplot(x= "x", y = "y",data= segs_with_dists.loc[lambda x: x.transformed_dist > .5], s=2,alpha=.2,edgecolor=None)
rscl= 1
sns.scatterplot(x= "x", y = "y",data= segs_with_dists.loc[lambda x: x.transformed_dist < .5],alpha=.2,edgecolor=None, s=10,)
f = plt.gcf()
f.set_size_inches(5,5)
plt.gca().set_xlim([0,6])
plt.gca().set_ylim([0,6])
ax = plt.gca()


In [9]:
## f = plt.gcf()
f.set_size_inches(12,12)
sns.heatmap((winning_pixels.unstack("y").T.sort_index().T- best_seg.min().min()).fillna(0))

ax = plt.gca()


In [30]:

winning_pixels.rename("segment").to_frame().to_csv(winning_pixels_file(dsname))
passing_pixels.rename("kde_val").to_frame().to_csv(passing_pixels_file(dsname))



In [31]:
winning_pixels2 = pd.read_csv(winning_pixels_file(dsname),index_col=["x","y"])
passing_pixels2 = pd.read_csv(passing_pixels_file(dsname),index_col=["x","y","g"])

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [33]:
print(winning_pixels_file(dsname))

None


In [10]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax=None, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)
    
    if ax is None:
        ellipse.set_transform(transf) 
    else:
        ellipse.set_transform(transf+ ax.transData)
            
        
    return ellipse
    #return ax.add_patch(ellipse)